In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Importing data

In [4]:
# Dataset folders 
dataset_dir = '/kaggle/input/an2dl-challenge-1-nt/data_splitted_no_test'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

In [6]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator, and for the trainign with Data Augmentation
train_data_gen = ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        fill_mode='reflect',
                                        rescale=1/255.) # rescale value is multiplied to the image ############################################
valid_data_gen = ImageDataGenerator(rescale=1/255.)
# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=8,
                                                       shuffle=True,
                                                       seed=seed)

valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)

Found 3648 images belonging to 8 classes.
Found 648 images belonging to 8 classes.


## Model

In [7]:
# to create a folder
import os
directory = "MicheleNet1"
parent_dir = "/kaggle/working"
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [8]:
!cd /kaggle/working/MicheleNet1

In [9]:
input_shape = (96, 96, 3)
epochs = 200

In [10]:
def michelenet1(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    
    #x = tfkl.Resizing(64, 64, interpolation="bicubic", name='resizing')(input_layer)
    
    conv0 = tfkl.Conv2D(
        filters=256,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(input_layer)
    
    conv1 = tfkl.Conv2D(
        filters=512,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(conv0)
    pool1 = tfkl.MaxPooling2D()(conv1)

    x = tfkl.GlobalAveragePooling2D(name='gap')(pool1)
    classifier_layer = tfkl.Dense(units=1024, name='Classifier', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(x)
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.RMSprop(0.0001), metrics='accuracy')

    # Return the model
    return model

## Training

In [11]:
# Build model (for NO augmentation training)
MN1 = michelenet1(input_shape)
MN1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 256)       7168      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 512)       1180160   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 512)       0         
_________________________________________________________________
gap (GlobalAveragePooling2D) (None, 512)               0         
_________________________________________________________________
Classifier (Dense)           (None, 1024)              525312    
_________________________________________________________________
output_layer (Dense)         (None, 8)                 8200  

In [12]:
# Train the model
MN1_history = MN1.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)],
).history

Epoch 1/200
456/456 [==============================] - 34s 58ms/step - loss: 1.9858 - accuracy: 0.2072 - val_loss: 1.8539 - val_accuracy: 0.2454
Epoch 2/200
456/456 [==============================] - 20s 44ms/step - loss: 1.8339 - accuracy: 0.2741 - val_loss: 1.7594 - val_accuracy: 0.3164
Epoch 3/200
456/456 [==============================] - 20s 44ms/step - loss: 1.7430 - accuracy: 0.3235 - val_loss: 1.6760 - val_accuracy: 0.3302
Epoch 4/200
456/456 [==============================] - 20s 45ms/step - loss: 1.6680 - accuracy: 0.3646 - val_loss: 1.6023 - val_accuracy: 0.3904
Epoch 5/200
456/456 [==============================] - 20s 44ms/step - loss: 1.6132 - accuracy: 0.3901 - val_loss: 1.6267 - val_accuracy: 0.3441
Epoch 6/200
456/456 [==============================] - 21s 46ms/step - loss: 1.5633 - accuracy: 0.4178 - val_loss: 1.5150 - val_accuracy: 0.4228
Epoch 7/200
456/456 [==============================] - 21s 46ms/step - loss: 1.5143 - accuracy: 0.4435 - val_loss: 1.5393 - val_ac

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(MN1_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(MN1_history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(MN1_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(MN1_history['val_accuracy'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()